## Environemen Setup 

In [19]:
!curl ipinfo.io

{
  "ip": "104.196.231.123",
  "hostname": "123.231.196.104.bc.googleusercontent.com",
  "city": "The Dalles",
  "region": "Oregon",
  "country": "US",
  "loc": "45.5946,-121.1787",
  "org": "AS15169 Google LLC",
  "postal": "97058",
  "timezone": "America/Los_Angeles",
  "readme": "https://ipinfo.io/missingauth"
}

In [22]:
!nvidia-smi

Mon May 24 07:23:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/auspicious3000/autovc.git
%cd autovc
!pip install wavenet_vocoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/My Drive/KTH/DT2119/autovc/auth.json' 
!echo $GOOGLE_APPLICATION_CREDENTIALS

project='dt2119-autovc' # change to your project name here
os.environ['GCP_PROJECT'] = project 
os.environ['GCP_ACCOUNT'] = 'bdcc-colab@' + project + '.iam.gserviceaccount.com'

!gcloud auth activate-service-account "$GCP_ACCOUNT" --key-file="$GOOGLE_APPLICATION_CREDENTIALS" --project="$GCP_PROJECT"

/content/drive/My Drive/KTH/DT2119/autovc/auth.json
Activated service account credentials for: [bdcc-colab@dt2119-autovc.iam.gserviceaccount.com]


To take a quick anonymous survey, run:
  $ gcloud survey



In [17]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   137k      0 --:--:-- --:--:-- --:--:--  145k
OK
74 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 74 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [20]:
!mkdir DataSet
!gcsfuse autovc_datasets DataSet

2021/05/24 07:04:32.891058 Using mount point: /content/autovc/DataSet
2021/05/24 07:04:32.897752 Opening GCS connection...
2021/05/24 07:04:33.118541 Mounting file system "autovc_datasets"...
2021/05/24 07:04:33.119050 File system has been successfully mounted.


## Test Pretrained AUTOVC

In [24]:
PATH = "/content/drive/MyDrive/KTH/DT2119/autovc/"

In [ ]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
from tqdm import tqdm


def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

device = 'cuda:0'
G = Generator(32,256,512,32).eval().to(device)

g_checkpoint = torch.load(PATH + 'autovc.ckpt', map_location=device)
G.load_state_dict(g_checkpoint['model'])

metadata = pickle.load(open(PATH + 'inference_data_vctk.pkl', "rb"))

spect_vc = []

for sbmt_i in tqdm(metadata):
             
    x_org = sbmt_i[2]
    x_org, len_pad = pad_seq(x_org)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(sbmt_i[1][np.newaxis, :]).to(device)
    
    for sbmt_j in metadata:
                   
        emb_trg = torch.from_numpy(sbmt_j[1][np.newaxis, :]).to(device)
        
        with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
            
        if len_pad == 0:
            uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
        else:
            uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
        
        spect_vc.append( ('{}x{}'.format(sbmt_i[0], sbmt_j[0]), uttr_trg) )
        
        
with open(PATH + 'results.pkl', 'wb') as handle:
    pickle.dump(spect_vc, handle)

 57%|█████▋    | 57/100 [02:50<02:09,  3.01s/it]

In [37]:
import pickle
results = pickle.load(open('metadata.pkl', "rb"))
results[0][2].shape

(90, 80)

In [31]:
metadata = pickle.load(open(PATH + 'vctk_inference_data.pkl', "rb"))
# metadata

In [46]:
metadata = pickle.load(open(PATH + 'inference_data_vctk.pkl', "rb"))
metadata[2][1].shape

(256,)

In [6]:
import torch
import librosa
import pickle
from synthesis import build_model
from synthesis import wavegen
import soundfile as sf

spect_vc = pickle.load(open('results.pkl', 'rb'))
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load(PATH+"checkpoint_step001000000_ema.pth", map_location=device)
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)   
    sf.write("wavs/" + name+'.wav', waveform, samplerate=16000)

  0%|          | 0/23040 [00:00<?, ?it/s]

p225xp225


  0%|          | 13/23040 [00:00<03:04, 124.80it/s]

p225xp228


  0%|          | 13/23040 [00:00<03:04, 124.94it/s]

...

  0%|          | 13/27904 [00:00<03:51, 120.64it/s]

p270xp256


  0%|          | 13/27904 [00:00<03:48, 121.83it/s]

p270xp270


100%|██████████| 27904/27904 [03:55<00:00, 118.57it/s]
